# pyobsplot

**Note :** this notebook is designed to be used on [Google Colab](https://colab.research.google.com/github/juba/pyobsplot/blob/main/examples/introduction.ipynb).

[pyobsplot](https://github.com/juba/pyobsplot) is a Python package which allows to use Observable Plot in Jupyter notebooks with a syntax as close as possible to the JavaScript one. For more information, see the [documentation website](https://juba.github.io/pyobsplot).

## Getting started

First, we install the package:

In [ ]:
# Only needed in Colab, cleanup environment
! pip uninstall -y pandas-gbq
# Install pyobsplot
! pip install pyobsplot

Then we can import the needed modules and functions:

In [ ]:
from pyobsplot import Obsplot, Plot, d3, js
import polars as pl

We can then generate our first plots:

In [ ]:
penguins = pl.read_csv("https://github.com/juba/pyobsplot/raw/main/doc/data/penguins.csv")

Obsplot({
    "grid": True,
    "marks": [
        Plot.dot(penguins, {"x": "culmen_length_mm", "y": "culmen_depth_mm", "fill": "island"})
    ]
})

In [ ]:
Obsplot({
  "grid": True,
  "marginRight": 80,
  "color": {"legend": True},
  "marks": [
    Plot.rectY(penguins, Plot.binX({"y": "count"}, {"x": "body_mass_g", "fill": "island", "fy": "island"})),
    Plot.ruleY([0])
  ]
})

## jsdom renderer

By default, plots are created as Jupyter widgets, using the `widget` renderer. `pyobsplot` provides another renderer, called `jsdom`, which generates plots directly as SVG or HTML.

To use this renderer, we need to have a working installation of a recent node.js version:

In [ ]:
! curl -fsSL https://deb.nodesource.com/setup_18.x | bash - &&\
! apt-get install -y nodejs

And we need to install the `pyobsplot` npm package:

In [ ]:
! npm install pyobsplot

Then we can switch to the `jsdom` renderer:

In [ ]:
Obsplot.set_renderer("jsdom")

After that, the following plots will be generated directly as SVG or HTML.

In [ ]:
from datetime import datetime

ipos = pl.read_csv("https://github.com/juba/pyobsplot/raw/main/doc/data/ipos.csv", try_parse_dates=True).filter(
    pl.col("date") > datetime(1991, 1, 1)
)


Obsplot({
    "insetRight": 10,
    "height": 600,
    "width": 600,
    "marks": [
        Plot.dot(
            ipos, Plot.dodgeY({"x": "date", "r": "rMVOP", "fill": "currentColor"})
        ),
        Plot.text(
            ipos,
            Plot.dodgeY(
                {
                    "filter": js("(d) => d.rMVOP > 5e3"),
                    "x": "date",
                    "r": "rMVOP",
                    "text": js("d => (d.rMVOP / 1e3).toFixed()"),
                    "fill": "white",
                    "fontWeight": "bold",
                }
            ),
        ),
    ],
})

In [ ]:
ca55 = pl.read_csv("https://github.com/juba/pyobsplot/raw/main/doc/data/ca55-south.csv")

Obsplot({
    "x": {"axis": None},
    "y": {"axis": None},
    "inset": 10,
    "marginBottom": 2,
    "height": 500,
    "color": {"type": "diverging"},
    "marks": [
        Plot.raster(
            ca55,
            {
                "x": "LONGITUDE",
                "y": "LATITUDE",
                "fill": "MAG_IGRF90",
                "interpolate": "random-walk",
            },
        ),
        Plot.frame(),
    ],
})
